### Testing New csv Load In Method

In [14]:
import boto3
bucket='yelpchallenge1' # Or whatever you called your bucket
data_key = 'datasets/df.csv' # Where the file is within your bucket
data_location = 's3://{}/{}'.format(bucket, data_key)
df = pd.read_csv(data_location)

KeyboardInterrupt: 

In [13]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import theano
import spacy
nlp = spacy.load('en')

In [ ]:
# import final .csv
df = pd.read_csv('s3://yelpchallenge1/datasets/df.csv')

In [4]:
# check import
df.head()

,Unnamed: 0,stars,text,date,total_votes,tokens
0,0,1,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,7.0,"['total', 'horrible', 'service', 'crooks', 'ac..."
1,1,5,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,0.0,"['adore', 'travis', 'hard', 'rock', 'kelly', '..."
2,2,5,I have to say that this office really has it t...,2016-11-09 20:09:03,3.0,"['office', 'organized', 'friendly', 'phillipp'..."
3,3,5,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,0.0,"['went', 'lunch', 'steak', 'sandwich', 'delici..."
4,4,1,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,7.0,"['today', 'second', 'sessions', 'paid', 'sessi..."


In [5]:
# check datatypes of columns in dataframe
df.dtypes

Unnamed: 0      object
stars           object
text            object
date            object
total_votes    float64
tokens          object
dtype: object

In [6]:
# drop unwanted columns
df = df.drop(columns=['Unnamed: 0'])

In [7]:
df.head()

,stars,text,date,total_votes,tokens
0,1,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36,7.0,"['total', 'horrible', 'service', 'crooks', 'ac..."
1,5,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33,0.0,"['adore', 'travis', 'hard', 'rock', 'kelly', '..."
2,5,I have to say that this office really has it t...,2016-11-09 20:09:03,3.0,"['office', 'organized', 'friendly', 'phillipp'..."
3,5,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38,0.0,"['went', 'lunch', 'steak', 'sandwich', 'delici..."
4,1,Today was my second out of three sessions I ha...,2018-01-30 23:07:38,7.0,"['today', 'second', 'sessions', 'paid', 'sessi..."


In [8]:
df.dtypes

stars           object
text            object
date            object
total_votes    float64
tokens          object
dtype: object

In [65]:
# create mini-dataframe for testing
# want to make sure works locally on small dataset before scaling to entire dataset/AWS
mini_df = df.head(10)

In [ ]:
# exploring TFIDF Vectorizer

In [50]:
# new tokenizer
def tokenize(document):
    doc = nlp(document)
    return [token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [69]:
# create variable to feed into TFIDF Vectorizer fit_transform
# to be updated to 'text' column of main dataframe (df['text']) for vectorization in AWS
data = mini_df['text']

In [72]:
data[2]

"I have to say that this office really has it together, they are so organized and friendly!  Dr. J. Phillipp is a great dentist, very friendly and professional.  The dental assistants that helped in my procedure were amazing, Jewel and Bailey helped me to feel comfortable!  I don't have dental insurance, but they have this insurance through their office you can purchase for $80 something a year and this gave me 25% off all of my dental work, plus they helped me get signed up for care credit which I knew nothing about before this visit!  I highly recommend this office for the nice synergy the whole office has!"

In [71]:
# vectorizer
tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))

# Create a vocabulary and get word counts per document
# Learn vocab and transform data into form we want
sparse = tfidf.fit_transform(data)

# Print word counts

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head(50)

,,,ask,awesome,be,big,clearly,come,dental,dr,...,wrong guest,year,year add,year admit,year ago,year come,year food,year give,zucchini,zucchini appetizer
0,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
1,0.382096,0.000000,0.000000,0.05084,0.000000,0.000000,0.000,0.05084,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
2,0.199434,0.000000,0.000000,0.00000,0.000000,0.000000,0.000,0.00000,0.185752,0.092876,...,0.000000,0.061412,0.000000,0.000000,0.000000,0.000000,0.000,0.092876,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000,0.00000,0.000000,0.000000,...,0.036283,0.023992,0.000000,0.036283,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
5,0.065694,0.000000,0.061187,0.00000,0.061187,0.000000,0.000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.061187,0.061187
6,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000,0.00000,0.000000,0.000000,...,0.000000,0.119463,0.060223,0.000000,0.060223,0.060223,0.000,0.000000,0.000000,0.000000
7,0.140649,0.000000,0.000000,0.00000,0.000000,0.000000,0.131,0.00000,0.000000,0.000000,...,0.000000,0.086621,0.000000,0.000000,0.000000,0.000000,0.131,0.000000,0.000000,0.000000
8,0.034224,0.000000,0.000000,0.00000,0.000000,0.000000,0.000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
9,0.266859,0.248551,0.000000,0.00000,0.000000,0.248551,0.000,0.00000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000
